In [1]:
from datetime import datetime
import pandas as pd
import warnings
import os

warnings.filterwarnings("ignore")

In [2]:
####################################### ACTUALIZAR CARTERA CCD #######################################

In [3]:
# Depurar Nuevo_DACxANALISTA
columnas = ['DEUDOR', 'NOMBRE', 'RUC', 'ANALISTA_ACT', 'TIPO_DAC', 'ESTADO']
no_analistas = ['REGION NORTE', 'REGION SUR', 'WALTER LOPEZ', 'SIN INFORMACION']
no_estados = ['LIQUIDADO', 'PROCESO DE LIQUIDACIÓN']

df_dacx = pd.read_excel('docs/Nuevo_DACxANALISTA.xlsx', sheet_name='Base_NUEVA', usecols=columnas)
df_dacx = df_dacx.rename(columns={'ANALISTA_ACT': 'ANALISTA', 'TIPO_DAC': 'TIPO'})
df_dacx = df_dacx[~df_dacx['ANALISTA'].isin(no_analistas)]
df_dacx = df_dacx[~df_dacx['ESTADO'].isin(no_estados)]
df_dacx['DEUDOR'] = df_dacx['DEUDOR'].astype("Int64").astype(str)
df_dacx['RUC'] = df_dacx['RUC'].astype("Int64").astype(str)
df_dacx.dropna(subset=['DEUDOR'], inplace=True)
df_dacx.drop_duplicates(subset=['DEUDOR'], inplace=True)
df_dacx.reset_index(drop=True, inplace=True)

list_analistas = df_dacx['ANALISTA'].unique() # Lista de analistas

# Actualizar CARTERA_GENERAL_CCD
df_cartera = pd.read_excel('docs/CARTERA_CCD.xlsx', sheet_name='GENERAL', usecols=['DEUDOR', 'APOYO1', 'APOYO2', 'APOYO3'])
df_cartera['DEUDOR'] = df_cartera['DEUDOR'].astype("Int64").astype(str)
df_cartera = pd.merge(df_cartera, df_dacx, on='DEUDOR', how='right')
df_cartera = df_cartera[['DEUDOR', 'NOMBRE', 'ANALISTA', 'APOYO1', 'APOYO2', 'APOYO3', 'ESTADO', 'TIPO']]
df_cartera.dropna(subset=['DEUDOR'], inplace=True)
df_cartera.drop_duplicates(subset=['DEUDOR'], inplace=True)
df_cartera.sort_values(by='DEUDOR', inplace=True, ignore_index=True)
df_cartera.reset_index(drop=True, inplace=True)
df_cartera.to_excel('docs/CARTERA_CCD.xlsx', sheet_name='GENERAL', index=False)

# Crear hojas por analista
with pd.ExcelWriter('docs/CARTERA_CCD.xlsx') as writer:
    df_temp = df_cartera.copy()
    df_cartera.to_excel(writer, sheet_name='GENERAL', index=False)
    for analista in list_analistas:
        df_cartera = df_temp[df_temp['ANALISTA'] == analista]
        df_cartera.sort_values(by='DEUDOR', inplace=True, ignore_index=True)
        df_cartera.reset_index(drop=True, inplace=True)
        print(f'HOJA: {analista} [{df_cartera.shape[0]}]')
        df_cartera.to_excel(writer, sheet_name=analista, index=False)

HOJA: DIEGO RODRIGUEZ [85]
HOJA: JOSE LUIS VALVERDE [83]
HOJA: JUAN CARLOS HUATAY [76]
HOJA: YOLANDA OLIVA [78]
HOJA: RAQUEL CAYETANO [81]


In [4]:
####################################### BASE CELULARES vs CARTERA CCD #######################################

In [5]:
df_celulares = pd.read_excel('docs/Base_Celulares_CCD.xlsx')
df_celulares['DEUDOR'] = df_celulares['DEUDOR'].astype("Int64").astype(str)
df_celulares['CELULAR'] = df_celulares['CELULAR'].astype("Int64").astype(str)
df_celulares = df_celulares[['DEUDOR', 'CELULAR']]
df_celulares['CELULAR'] = df_celulares[df_celulares['CELULAR'].str.len() == 9]['CELULAR']
df_celulares.dropna(subset=['CELULAR'], inplace=True)
df_celulares['CELULAR'] = '51' + df_celulares['CELULAR']
df_celulares.reset_index(drop=True, inplace=True)

df_cartera = pd.read_excel('docs/CARTERA_CCD.xlsx', sheet_name='GENERAL', usecols=['DEUDOR', 'ANALISTA', 'APOYO1', 'APOYO2', 'APOYO3'])
df_cartera['DEUDOR'] = df_cartera['DEUDOR'].astype("Int64").astype(str)
df_cruce = pd.merge(df_celulares, df_cartera, on='DEUDOR', how='left')
df_cruce.dropna(subset=['ANALISTA'], inplace=True)
df_cruce.dropna(subset=['CELULAR'], inplace=True)
df_cruce.reset_index(drop=True, inplace=True)
df_cruce.head(5)

,DEUDOR,CELULAR,ANALISTA,APOYO1,APOYO2,APOYO3
0,4070709,51997108820,YOLANDA OLIVA,DIEGO RODRIGUEZ,JOSE LUIS VALVERDE,JUAN CARLOS HUATAY
1,4071078,51997108224,JOSE LUIS VALVERDE,DIEGO RODRIGUEZ,YOLANDA OLIVA,RAQUEL CAYETANO
2,4070112,51980588839,JUAN CARLOS HUATAY,JOSE LUIS VALVERDE,RAQUEL CAYETANO,YOLANDA OLIVA
3,4070230,51950304684,YOLANDA OLIVA,RAQUEL CAYETANO,DIEGO RODRIGUEZ,JOSE LUIS VALVERDE
4,4070266,51979717800,YOLANDA OLIVA,JOSE LUIS VALVERDE,RAQUEL CAYETANO,DIEGO RODRIGUEZ


In [6]:
####################################### CADENA DE TEXTO #######################################

In [7]:
# (f'{row["CELULAR"]}{texto1}{row["ANALISTA"]}{texto2}{row["FECHA_RETORNO"]}{texto3}{row["APOYO"]}{texto4}')
df_regla = pd.read_excel('docs/REGLA.xlsx', sheet_name='TEXTO')
texto1 = df_regla['TEXTO_1'][0]
texto2 = df_regla['TEXTO_2'][0]
texto3 = df_regla['TEXTO_3'][0]
texto4 = df_regla['TEXTO_4'][0]
print(f'{texto1}\n{texto2}\n{texto3}\n{texto4}')

,Text,"Estimado Socio: Le informamos que su analista 
, se encontra de vacaciones hasta el dia 
. Quien quedara a cargo de la cuenta comercial sera el/la analista 
. Gracias por su comprension. Creditos y Cobranzas Distribuidores le desea un excelente dia"


In [8]:
####################################### ANALISTAS DE VACACIONES #######################################

In [9]:
fecha_hoy = datetime.now().strftime('%d.%m.%Y')

df_vacaciones = pd.read_excel('docs/VACACIONES.xlsx')
df_vacaciones.dropna(inplace=True)
dict_analistas = dict(zip(df_vacaciones['ANALISTA'], zip(df_vacaciones['FECHA_SALIDA'], df_vacaciones['FECHA_RETORNO'])))

vacaciones = {}
for analista, fechas in dict_analistas.items():
    if fechas[0] <= fecha_hoy <= fechas[1]:
        print(f'{analista} de vacaciones desde {fechas[0]} hasta {fechas[1]}')
        vacaciones.update({analista: fechas[1]})
vacaciones

JUAN CARLOS HUATAY de vacaciones desde 29.05.2024 hasta 31.05.2024


{'JUAN CARLOS HUATAY': '31.05.2024'}

In [10]:
####################################### AGREGAR COLUMNA FECHA_RETORNO Y APOYO #######################################

In [11]:
analistas = list(vacaciones.keys())
df_cruce = df_cruce[df_cruce['ANALISTA'].isin(analistas)]
df_cruce['FECHA_RETORNO'] = df_cruce['ANALISTA'].map(vacaciones)
df_cruce['APOYO'] = df_cruce[['APOYO1', 'APOYO2', 'APOYO3']].apply(
    lambda x: x[0] if x[0] not in (analistas) 
    else (x[1] if x[1] not in (analistas) else x[2]), axis=1)
df_cruce['FECHA_RETORNO'] = df_cruce['FECHA_RETORNO'].str.replace('.', '/')
df_cruce.reset_index(drop=True, inplace=True)
df_cruce = df_cruce[['CELULAR', 'ANALISTA', 'FECHA_RETORNO','APOYO']]
df_cruce

,CELULAR,ANALISTA,FECHA_RETORNO,APOYO
0,51980588839,JUAN CARLOS HUATAY,31/05/2024,JOSE LUIS VALVERDE
1,51966773333,JUAN CARLOS HUATAY,31/05/2024,JOSE LUIS VALVERDE
2,51931872630,JUAN CARLOS HUATAY,31/05/2024,DIEGO RODRIGUEZ
3,51993963502,JUAN CARLOS HUATAY,31/05/2024,JOSE LUIS VALVERDE
4,51982481201,JUAN CARLOS HUATAY,31/05/2024,YOLANDA OLIVA
5,51940293023,JUAN CARLOS HUATAY,31/05/2024,DIEGO RODRIGUEZ
6,51978098070,JUAN CARLOS HUATAY,31/05/2024,JOSE LUIS VALVERDE
7,51943831053,JUAN CARLOS HUATAY,31/05/2024,YOLANDA OLIVA
8,51913046862,JUAN CARLOS HUATAY,31/05/2024,DIEGO RODRIGUEZ
9,51945312908,JUAN CARLOS HUATAY,31/05/2024,DIEGO RODRIGUEZ


In [12]:
####################################### AGREGAR COLUMNA TEXTO #######################################

In [13]:
# (f'{row["CELULAR"]}{texto1}{row["ANALISTA"]}{texto2}{row["FECHA_RETORNO"]}{texto3}{row["APOYO"]}{texto4}')
df_cruce['TEXTO'] = df_cruce.apply(lambda row: f'{row["CELULAR"]}{texto1}{row["ANALISTA"]}{texto2}{row["FECHA_RETORNO"]}{texto3}{row["APOYO"]}{texto4}', axis=1)
df_cruce

,CELULAR,ANALISTA,FECHA_RETORNO,APOYO,TEXTO
0,51980588839,JUAN CARLOS HUATAY,31/05/2024,JOSE LUIS VALVERDE,"51980588839,Text,""Estimado Socio: Le informamo..."
1,51966773333,JUAN CARLOS HUATAY,31/05/2024,JOSE LUIS VALVERDE,"51966773333,Text,""Estimado Socio: Le informamo..."
2,51931872630,JUAN CARLOS HUATAY,31/05/2024,DIEGO RODRIGUEZ,"51931872630,Text,""Estimado Socio: Le informamo..."
3,51993963502,JUAN CARLOS HUATAY,31/05/2024,JOSE LUIS VALVERDE,"51993963502,Text,""Estimado Socio: Le informamo..."
4,51982481201,JUAN CARLOS HUATAY,31/05/2024,YOLANDA OLIVA,"51982481201,Text,""Estimado Socio: Le informamo..."
5,51940293023,JUAN CARLOS HUATAY,31/05/2024,DIEGO RODRIGUEZ,"51940293023,Text,""Estimado Socio: Le informamo..."
6,51978098070,JUAN CARLOS HUATAY,31/05/2024,JOSE LUIS VALVERDE,"51978098070,Text,""Estimado Socio: Le informamo..."
7,51943831053,JUAN CARLOS HUATAY,31/05/2024,YOLANDA OLIVA,"51943831053,Text,""Estimado Socio: Le informamo..."
8,51913046862,JUAN CARLOS HUATAY,31/05/2024,DIEGO RODRIGUEZ,"51913046862,Text,""Estimado Socio: Le informamo..."
9,51945312908,JUAN CARLOS HUATAY,31/05/2024,DIEGO RODRIGUEZ,"51945312908,Text,""Estimado Socio: Le informamo..."


In [14]:
####################################### EXPORTAR TXT #######################################

In [15]:
sms_vacaciones = df_cruce["TEXTO"].to_list()
with open('sms_vacaciones.txt', "w") as f:
    for item in sms_vacaciones:
        f.write("%s\n" % item)

os.startfile('sms_vacaciones.txt')